In [ ]:
import torch
import framework
import math
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%config InlineBackend.print_figure_kwargs={'facecolor' : '#00000000'}

In [ ]:
def generate_disc_set(nb):
    # generate unif samples
    p = torch.FloatTensor(nb, 2).uniform_(0, 1)
    # assign label inside/outside circle
    center = torch.tensor([0.5,0.5]).float()
    inside = ( torch.norm(p-center, dim=1) < (1.0 / math.sqrt(2*math.pi)) )
    # Hot encoding
    label = torch.zeros(inside.shape[0], 2)
    label[inside,1] = 1
    label[-(inside-1),0] = 1
    return p, label

In [ ]:
N = 1000
x_train, y_train = generate_disc_set(N)
x_test, y_test = generate_disc_set(N)

In [ ]:
plt.axis([-2,2,-2,2])
plt.scatter(x_train[y_train[:,1] ==1][:,0], x_train[y_train[:,1] ==1][:,1])
plt.scatter(x_train[y_train[:,0] ==1][:,0], x_train[y_train[:,0] ==1][:,1])
plt.figure(figsize=(5,5), dpi=80)

In [ ]:
H = 25 # hidden layer size
hidden_nb = 3 # hidden layer amount
modules_array = []

# Input layer
modules_array.append(framework.Linear(2, H, 'relu'))
# Hidden layers
for i in range(0, hidden_nb):
    modules_array.append(framework.ReLU())
    modules_array.append(framework.Linear(H, H, 'relu'))
# Output layer
modules_array.append(framework.ReLU())
modules_array.append(framework.Linear(H, 2, 'none'))

model = framework.Sequential(modules_array)

In [ ]:
def train(model, x_train, y_train) :
    mini_batch_size = 100
    epochs = 10
    lr = 1e-1
    optimizer = framework.SGD(model.param(), lr)
    for e in range(0, epochs):
         for b in range(0, x_train.size(0), mini_batch_size):
                # Forward pass of neural net
                x_batch = x_train.narrow(0, b, mini_batch_size)
                y_batch = y_train.narrow(0, b, mini_batch_size)
                output = model.forward(x_batch)
                # Compute loss
                loss_module = framework.LossMSE(y_batch)
                loss = loss_module.forward(output)
                print(loss.item())
                # Set all gradients to zero
                optimizer.zero_grad()
                # Backward
                model.backward(loss_module.backward())
                # Step
                optimizer.step()
                

In [ ]:
train(model, x_train, y_train)

In [ ]:
#def test(model, x_test, y_test):
#    output = model.forward(x_test)
#    error_percent = arg

In [ ]:
#def compute_nb_errors(model, data_input, data_target) :
#    # Forward pass
#    output = model(data_input)
#    # Count errors
#    errors = (output.argmax(dim=1) != data_target)
#    return errors.sum().item()